In [16]:
import requests
import pandas as pd
import numpy as np
import time
import oracledb
import getpass
from geopy.geocoders import Nominatim

In [18]:
# Run indefinitely
while True:
    # Define functions for geolocation
    geolocator = Nominatim(user_agent="MU-DS-Capstone")
    def fix_address(address):
        #Error catching
        #Not-string error
        address = str(address)
        # Remove city
        if "," in address:
            address = address[:address.index(",")]
        # If intersection, just use first street. TODO: Improve this
        if "/" in address:
            address = address[:address.index("/")]
        # Remove NA values
        if pd.isna(address):
            address = np.nan
        # Change block address to just be that address number
        address = address.replace("-BLK", "")
        # Catching the BLVD error, needs to say BLVD not BL
        if address.endswith("BL"):
            address += "VD"
        # Catching MLK DR error, use old name
        address = address.replace("MARTIN L KING JR DR", "OLD WORLD THIRD ST")
        # Mc Kinley Error
        address = address.replace("MC KINLEY AV", "MCKINLEY AV")
        # Bluemound road error
        address = address.replace("BLUE MOUND RD", "BLUEMOUND RD")        
        return address + " MILWAUKEE"

    def get_gps(geocoded, index):
        if geocoded is None:
            return np.nan
        return geocoded[1][index]

    # Grab police call log from HTML table
    url = 'https://itmdapps.milwaukee.gov/MPDCallData/index.jsp?district=All'
    headers = {'User-Agent': 'Marquette Data Science'}
    html = requests.get(url, headers=headers).content
    df = pd.read_html(html, index_col=0, header=None)[-1]
    # Remove multi-index
    df.columns = df.columns.droplevel(0)
    # Remove duplicate entries
    df.drop_duplicates(inplace=True)

    # Add Latitude and Longitude columns to dataframe
    fixed_addresses = df.Location.transform(fix_address)
    geocoded_addresses = fixed_addresses.apply(geolocator.geocode)
    df["Latitude"] = geocoded_addresses.apply(get_gps, index=0)
    df["Longitude"] = geocoded_addresses.apply(get_gps, index=1)

    """ SEND DATA TO SQL DATABASE """
    
    time.sleep(30*60) # Sleep for 30 minutes

In [32]:
df

Call Number,Date/Time,Location,Police District,Nature of Call,Status,Latitude,Longitude
230310847,01/31/2023 12:58:13 PM,"3626 W FOND DU LAC AV,MKE",7,FOLLOW UP,Assignment Completed,43.076747,-87.958838
230310845,01/31/2023 12:57:27 PM,"245 W LINCOLN AV,MKE",2,ASSIGNMENT,Service in Progress,43.002518,-87.913685
230310805,01/31/2023 12:56:54 PM,"2336 N 38TH ST,MKE",3,STOLEN VEHICLE,Service in Progress,43.061377,-87.960735
230310556,01/31/2023 12:56:18 PM,"5110 W LOCUST ST,MKE",7,TRBL W/JUV,Service in Progress,43.072585,-87.977984
230310843,01/31/2023 12:56:02 PM,"906 W HISTORIC MITCHELL ST,MKE",2,COMMUNITY MTNG,Service in Progress,43.012466,-87.923022
...,...,...,...,...,...,...,...
230310853,01/31/2023 01:03:22 PM,"2333 N 49TH ST,MKE",3,INVESTIGATION,Service in Progress,43.061594,-87.975478
230310851,01/31/2023 01:02:19 PM,"6929 W SILVER SPRING DR,MKE",4,PRISONER TRANS,Service in Progress,43.119006,-87.998323
230310480,01/31/2023 01:01:51 PM,"5435 N 92ND ST,MKE",4,FAMILY TROUBLE,Service in Progress,43.117234,-88.028323
230310838,01/31/2023 01:00:29 PM,"N 25TH ST / W GARFIELD AV,MKE",3,LOOSE ANIMAL,Unable to Locate Complainant,43.036005,-87.945032


In [30]:
password = getpass.getpass("Enter DB Password")
wallet_password = getpass.getpass("Enter Wallet Password")

connection = oracledb.connect(
    user="admin",
    password=password,
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password=wallet_password)

print("Successfully connected to Oracle Database")
cursor = connection.cursor()

Enter DB Password ················
Enter Wallet Password ················


Successfully connected to Oracle Database
